In [3]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import pickle
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import random
import copulae
from copulae import EmpiricalCopula, pseudo_obs
from copulae.datasets import load_marginal_data
import yfinance as yf
from yahoofinancials import YahooFinancials
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.api import VAR
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression
import branca
from scipy.interpolate import griddata
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.collections import PatchCollection
import seaborn as sns
import tslearn

ImportError: cannot import name 'EmpiricalCopula' from 'copulae' (/Users/leiflyu/miniforge3/envs/tf/lib/python3.8/site-packages/copulae/__init__.py)

In [2]:
# dvmr: discounted variable mortgage rates
url = "https://www.rba.gov.au/statistics/tables/xls/f05hist.xls"
r = requests.get(url)
open('f05hist.xls', 'wb').write(r.content)
df = pd.read_excel('f05hist.xls', sheet_name='Data', skiprows=10)
df = df.rename(columns={"Series ID": "date"})
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df = df.loc[df.index >= '2004-07-01']
df_dvmr = df['FILRHLBVD']

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [ ]:
#cr: cash rate
url = "https://www.rba.gov.au/statistics/tables/xls/f01hist.xls"
r = requests.get(url)
open('f01hist.xls', 'wb').write(r.content)
df = pd.read_excel('f01hist.xls', sheet_name='Data', skiprows=10)
df = df.rename(columns={"Series ID": "date"})
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df = df.loc[df.index >= '2004-07-01']
df_cr = df['FIRMMCRT']


In [ ]:
def nominal_to_effective(nominal_rate, periods_per_year):
    """
    Converts a nominal annual interest rate to an effective annual interest rate
    for a given number of compounding periods per year.
    
    Args:
        nominal_rate (float): The nominal annual interest rate (as a decimal).
        periods_per_year (int): The number of compounding periods per year.
    
    Returns:
        The effective annual interest rate (as a decimal).
    """
    return ((1 + 0.01* nominal_rate / periods_per_year) ** periods_per_year -1)*100


In [ ]:
df_rates = pd.concat([df_cr, df_dvmr],axis=1)

df_rates = df_rates.rename(columns={"FIRMMCRT": "Cash Rates", "FILRHLBVD": "Discounted Lending Rates"})
df_rates['HEAS(PLS)'] = 5.25
df_rates.loc['2020-01-31':'2021-12-31', 'HEAS(PLS)'] = 4.5
df_rates.loc['2022-01-31':, 'HEAS(PLS)'] = 3.95
df_effective_rates = df_rates.apply(lambda x: nominal_to_effective(x, 12))


In [ ]:
cumprod = (1+df_rates.loc['2021-01-31':'2021-12-31', 'Discounted Lending Rates']/1200).cumprod().values[-1]
((cumprod)**(1/26)-1)*26

In [ ]:
df_effective_rates.plot(figsize=(10, 5))
plt.xlabel('Date')
plt.ylabel('Effective Interest Rates (%)')


In [ ]:

xls1 = pd.read_excel("f01hist.xls", sheet_name="Data", header=2)
xls1 = xls1.drop(xls1.index[0:8], axis=0)
xls1["Description"] = pd.to_datetime(xls1["Description"])
xls1["Description"] = xls1["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))

xls3 = pd.read_excel("f11hist-1969-2009.xls", sheet_name="Data", header=2)
xls3 = xls3.drop(xls3.index[0:8], axis=0)
xls3["Description"] = pd.to_datetime(xls3["Description"])
xls3["Description"] = xls3["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))

xls4 = pd.read_excel("f11hist.xls", sheet_name="Data", header=2)
xls4 = xls4.drop(xls4.index[0:8], axis=0)
xls4["Description"] = pd.to_datetime(xls4["Description"])
xls4["Description"] = xls4["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))

xls34 = pd.concat([xls3, xls4], axis=0)

xls5 = pd.read_excel("g01hist.xls", sheet_name="Data", header=2)
xls5 = xls5.drop(xls5.index[0:8], axis=0)
xls5["Description"] = pd.to_datetime(xls5["Description"])
xls5["Description"] = xls5["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))

xls6 = pd.read_excel("h01hist.xls", sheet_name="Data", header=2)
xls6 = xls6.drop(xls6.index[0:8], axis=0)
xls6["Description"] = pd.to_datetime(xls6["Description"])
xls6["Description"] = xls6["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))

xls7 = pd.read_excel("h03hist.xls", sheet_name="Data", header=2)
xls7 = xls7.drop(xls7.index[0:8], axis=0)
xls7["Description"] = pd.to_datetime(xls7["Description"])
xls7["Description"] = xls7["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))
# Download ASX data from Yahoo Finance

xls8 = pd.read_excel("f05hist.xls", sheet_name="Data", header=2)
xls8 = xls8.drop(xls8.index[0:8], axis=0)
xls8["Description"] = pd.to_datetime(xls8["Description"])
xls8["Description"] = xls8["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))

ASX200data = yf.download('^AXJO', start='1999-01-01', end='2021-06-01', interval = "1mo", progress=False)
ASX200data = ASX200data.reset_index()
ASX200data["Description"] = pd.to_datetime(ASX200data["Date"])
ASX200data["Description"] = ASX200data["Description"].dt.date.apply(lambda x: x.strftime("%Y-%m"))

macvar = pd.DataFrame().assign(
    Description=xls7.loc[
        ("2021-04" > xls7["Description"]) & (xls7["Description"] > "2004-06")
    ]["Description"]
)
macvar.reset_index(drop=True, inplace=True)

macvar = pd.merge(
    macvar,
    xls1[["Description", "Cash Rate Target; monthly average"]],
    on="Description",
    how="left",
)
macvar = pd.merge(
    macvar,
    xls34[["Description", "AUD/USD Exchange Rate; see notes for further detail."]],
    on="Description",
    how="left",
)
macvar = pd.merge(
    macvar,
    xls5[["Description", "Consumer price index; All groups"]],
    on="Description",
    how="left",
)

macvar = pd.merge(
    macvar,
    xls6[["Description", "Gross domestic product (GDP); Chain volume"]],
    on="Description",
    how="left",
)

macvar = pd.merge(
    macvar,
    xls7[["Description", "Retail sales; All industries; Current price"]],
    on="Description",
    how="left",
)

macvar = pd.merge(
    macvar,
    xls7[["Description", "Private dwelling approvals"]],
    on="Description",
    how="left",
)

macvar = pd.merge(
    macvar,
    ASX200data[["Description", "Adj Close"]],
    on="Description",
    how="left",
)

macvar = pd.merge(
    macvar,
    xls8[["Description", "Lending rates; Housing loans; Banks; Variable; Discounted; Owner-occupier"]],
    on="Description",
    how="left",
)


macvar = macvar.set_axis(
    ["date", "ir", "exr", "cpi", "gdp", "rs", "pda", "asx","dvmr"], axis=1, inplace=False
)
macvar["ir"] = macvar["ir"].astype(float, errors="raise")
macvar["exr"] = macvar["exr"].astype(float, errors="raise")
macvar["cpi"] = macvar["cpi"].astype(float, errors="raise")
macvar["gdp"] = macvar["gdp"].astype(float, errors="raise")
macvar["rs"] = macvar["rs"].astype(float, errors="raise")
macvar["pda"] = macvar["pda"].astype(float, errors="raise")
macvar["asx"] = macvar["asx"].astype(float, errors="raise")
macvar["dvmr"] = macvar["dvmr"].astype(float, errors="raise")


In [ ]:
macvar = macvar.set_index('date')
macvar.index = pd.to_datetime(macvar.index)
macvar.index = macvar.index.to_period('M')
macvar = macvar.resample('Q').mean()

macvar['ir'] = macvar['ir'] - 0.09
macvar 

In [ ]:
macvar_log = macvar.apply(lambda x: np.log(x))
macvar_log = macvar_log.reset_index()

macvar_log

In [ ]:

threshold = 0.1

def selectStationaySeries(variable_tar):
    if adfuller(variable_tar.dropna(how="all"))[1] < threshold:
        stationary_variable = variable_tar
        suffix =  "original"

    elif adfuller(variable_tar.diff().dropna(how="all"))[1] < threshold:
        stationary_variable = variable_tar.diff().dropna(how="all")
        suffix = "1storderdiff"
    
    elif adfuller(variable_tar.diff(2).dropna(how="all"))[1] < threshold:
        stationary_variable =variable_tar.diff(2).dropna(how="all")
        suffix = "halfyear"
    
        
    elif adfuller(variable_tar.diff(4).dropna(how="all"))[1] < threshold:
        stationary_variable =variable_tar.diff(4).dropna(how="all")
        suffix = "oneyear"
    
    elif adfuller(variable_tar.diff().diff().dropna(how="all"))[1] < threshold:
        stationary_variable = variable_tar.diff().diff().dropna(how="all")
        suffix = "2ndorderdiff"     

#     elif adfuller(variable_tar.diff(3).dropna(how="all"))[1] < threshold:
#         stationary_variable = variable_tar.pct_change(3).dropna(how="all")
#         suffix = "seasonaldiff"

#     elif adfuller(variable_tar.diff(12).dropna(how="all"))[1] < threshold:
#         stationary_variable = variable_tar.pct_change(12).dropna(how="all")
#         suffix = "annualdiff" 


    else:
        print("not found")
    return(pd.DataFrame(stationary_variable),suffix)


macvarst = pd.DataFrame({})
macvarst["date"]  = macvar_log["date"] 
# macvarst["ir"] = macvarst["ir"].diff()
    

    

In [ ]:
st_way = []
for vn in macvar_log.columns.values[2:-1]:
    df, way = selectStationaySeries(macvar_log[vn])
    print(vn,way)
    macvarst = pd.concat([macvarst, df], axis=1)
    st_way.append(way)

In [ ]:
macdata = macvarst.loc[("2021Q1" >= macvarst["date"]) & (macvarst["date"] > "2004Q4")]

X_train = macdata.loc[:, macdata.columns != "date"]
scaler_pca_nat = StandardScaler()
X_train_std = scaler_pca_nat.fit_transform(X_train)
# scaler_pca_nat.inverse_transform()

pca_nat = PCA()
pca_nat.fit(X_train_std)

# cumsum = np.cumsum(pca_nat.explained_variance_ratio_)
# d0 = np.argmax(cumsum >= 0.9) + 1
# pca_nat = PCA(n_components=d0)
# pca_nat.fit(X_train_std)
X_pca_d = pca_nat.transform(X_train_std)

# However, in general, it is not possible to fully recover the original dataset from a PCA-transformed dataset,
#as some information is lost during the dimensionality reduction process.

In [ ]:
PCnames = []

d = np.shape(X_pca_d)[1]

for i in range(1, d + 1, 1):
    PCnames.append(f"PC{i}")
    
PC_nat = pd.DataFrame(X_pca_d, columns=PCnames)

PC_nat['date'] = macdata['date'].values


In [ ]:
PC_nat

In [ ]:
 df_fast = macvar_log[["ir","dvmr"]].diff().iloc[2:,:].reset_index(drop = True)

In [ ]:
PC_nat = pd.concat([df_fast,PC_nat],axis = 1)

In [ ]:
PC_l0 = PC_nat.set_index('date')


In [ ]:
# PC_nat = macvarst[macvarst['date']>'2004Q4']

In [ ]:
with open('DataFrameDict_postcode.pickle', 'rb') as f:
     DataFrameDict_postcode =  pickle.load(f) 
        
aveHPI_lv3 = []
lst = ["postcode", "sa4", "state", "logHPI", "logHPIdiff"]
# Calling DataFrame constructor on list
for key in DataFrameDict_postcode.keys():
    df = pd.DataFrame([], columns=lst)
    df["Description"] = DataFrameDict_postcode[key]["value_at_date"].loc[
        ("2021-06" > DataFrameDict_postcode[key]["value_at_date"])
        & (DataFrameDict_postcode[key]["value_at_date"] > "1999-12")
    ]
    df["logHPI"] =np.log2(
        DataFrameDict_postcode[key]["Hedonic Home Value Index"].loc[
            ("2021-06" > DataFrameDict_postcode[key]["value_at_date"])
            & (DataFrameDict_postcode[key]["value_at_date"] > "1999-12")
        ]
    )
    df["postcode"] = key
    df["sa4"] = DataFrameDict_postcode[key]["sa4_name16"].loc[
        ( DataFrameDict_postcode[key]["postcode"] == key)]
    df["state"] = DataFrameDict_postcode[key]["state"].loc[
        ( DataFrameDict_postcode[key]["postcode"] == key)]
    
    df["logHPIdiff"] = df["logHPI"].diff(1)
    aveHPI_lv3.append(df)

aveHPIdf_lv3 = pd.concat(aveHPI_lv3)
aveHPIdf_lv3 = aveHPIdf_lv3.reset_index(drop=True)
aveHPIdf_lv3['date'] = pd.to_datetime(aveHPIdf_lv3['Description'],errors = 'coerce')
aveHPIdf_lv3['year'] = pd.DatetimeIndex(aveHPIdf_lv3['date']).year

aveHPIdf_lv3 = aveHPIdf_lv3[aveHPIdf_lv3['date'] > '2000-12-31']



avg_1 = aveHPIdf_lv3.groupby(["date"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_1 = avg_1.rename(columns = {'mean':'l0'}).dropna().reset_index(drop = True)

sum_1 = aveHPIdf_lv3.groupby(["date"])["logHPIdiff"].agg(["sum"]).reset_index()
sum_1 = sum_1.rename(columns = {'sum':'l0_s'}).dropna().reset_index(drop = True)
sum_1 = sum_1.reset_index(drop=True)

#sa4 
avg_2 = aveHPIdf_lv3.groupby(["sa4","date"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_2 = avg_2.rename(columns = {'mean':'l01'}).dropna().reset_index(drop = True)

sum_2 = aveHPIdf_lv3.groupby(["sa4","date"])["logHPIdiff"].agg(["sum"]).reset_index()
sum_2 = sum_2.rename(columns = {'sum':'l01_s'}).dropna().reset_index(drop = True)
sum_2 = sum_2.reset_index(drop=True)

#postcode
avg_3 = aveHPIdf_lv3.groupby(["postcode","date"])["logHPIdiff"].agg(["mean"]).reset_index()
avg_3 = avg_3.rename(columns = {'mean':'l012'}).dropna().reset_index(drop = True)

# Set the 'date' column to a datetime format
avg_2['date'] = pd.to_datetime(avg_2['date'], format='%Y-%m-%d')

# Create a new dataframe with monthly dates from '2000-01-31' to '2021-05-31'
date_range = pd.date_range('2004-09-30', '2021-03-31', freq='M')

new_df = pd.DataFrame({'date': date_range})

# Use pivot to reshape the 'avg_2' dataframe
pivot_df = avg_2.pivot(index='date', columns='sa4', values='l01')

# Merge the 'new_df' and 'pivot_df' dataframes
merged_df = new_df.merge(pivot_df, on='date', how='outer')

# Sort the columns alphabetically by column name
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)

# Fill missing values with NaN
avg_2_pivot = merged_df.fillna(np.nan).drop('date', axis=1)

# Set the 'date' column to a datetime format
sum_2['date'] = pd.to_datetime(sum_2['date'], format='%Y-%m-%d')


new_df = pd.DataFrame({'date': date_range})

# Use pivot to reshape the 'sum_2' dataframe
pivot_df = sum_2.pivot(index='date', columns='sa4', values='l01_s')

# Merge the 'new_df' and 'pivot_df' dataframes
merged_df = new_df.merge(pivot_df, on='date', how='outer')

# Sort the columns alphabetically by column name
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)

# Fill missing values with NaN
sum_2_pivot = merged_df.fillna(np.nan).drop('date', axis=1)

# Set the 'date' column to a datetime format
avg_3['date'] = pd.to_datetime(avg_3['date'], format='%Y-%m-%d')

new_df = pd.DataFrame({'date': date_range})

# Use pivot to reshape the 'avg_3' dataframe
pivot_df = avg_3.pivot(index='date', columns='postcode', values='l012')

# Merge the 'new_df' and 'pivot_df' dataframes
merged_df = new_df.merge(pivot_df, on='date', how='outer')

# Sort the columns alphabetically by column name
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)

# Fill missing values with NaN
avg_3_pivot = merged_df.fillna(np.nan).drop('date', axis=1)

avg1 = avg_1.set_index("date")
avg2 = avg_2.set_index("date")
avg3 = avg_3.set_index("date")
df_dis =  pd.concat([avg1 , avg2.pivot(columns='sa4', values='l01'), avg3.pivot(columns='postcode', values='l012')],axis = 1)




In [ ]:
df_dis.index = df_dis.index.to_period('M')
df_dis = df_dis.loc['2005-01': '2021-03'].resample('Q').sum()

In [ ]:
# simplified 
def transform_df(df0):
    df = df0.copy()
     # subtract the value of each column in group 3 by the value of the corresponding column in group 2   
    for col in avg_3_pivot.columns:
        df[col] = df[col].sub(df['l0'], axis=0)
    return df.drop(avg_2_pivot, axis=1)


In [ ]:
df_l = transform_df(df_dis)

In [ ]:
train_data_nat= pd.merge(df_l['l0'], PC_l0, left_index=True, right_index=True)

In [ ]:
train_data_nat

In [ ]:
best_aic = float('inf')
best_model = None
for i in range(1, 5):
    model = VAR(train_data_nat)
    results = model.fit(i)
    print( results.aic)
    if results.aic < best_aic:
        best_aic = results.aic
        best_model = results
print(f'Best model has {best_model.k_ar} lags with AIC of {best_aic}')

In [ ]:
best_bic = float('inf')
best_model = None
for i in range(1, 5):
    model = VAR(train_data_nat)
    results = model.fit(i)
    print( results.bic)
    if results.bic < best_bic:
        best_bic = results.bic
        best_model = results
print(f'Best model has {best_model.k_ar} lags with BIC of {best_bic}')


In [ ]:
from dateutil.relativedelta import relativedelta

df_onesteperror = pd.DataFrame({},columns = avg_3_pivot.columns)
for pc in avg_3_pivot.columns:
    start_period = pd.Period('2005Q1', freq='Q')
    end_period = pd.Period('2017Q3', freq='Q')
    test_period = pd.Period('2017Q4', freq='Q')
    # Define a list to store the test dates
    errors = [] 
    # Repeat the simulation for subsequent test dates
    while test_period <=  pd.Period('2021Q1', freq='Q'):
        train_data =train_data_nat.loc[start_period:end_period]
        model = VAR(train_data)
        results = model.fit(1)
        X_test = results.forecast(train_data.values[-1:], steps=1)
        predicted_l0 = X_test[0][0]
        traningY = df_l.loc[start_period:end_period][pc]
        trainingdf1 = PC_l0.loc[start_period:end_period].reset_index(drop=True)
        Y = traningY.values
        X = trainingdf1.values
        X_train = sm.add_constant(X)
        model_full = sm.OLS(Y,X_train)
        reg_full = model_full.fit()
        predicted_l2 = reg_full.predict(X_test) 
        error = df_l.loc[test_period][pc]+train_data_nat.loc[test_period]['l0'] - (predicted_l0 + predicted_l2[0] )
        errors.append(error)
        start_period += 1
        end_period += 1
        # Update test period by one month
        test_period += 1
    df_onesteperror[pc] = errors

In [ ]:
df_emp = df_onesteperror
u = pseudo_obs(df_emp)
emp_cop = EmpiricalCopula(u, smoothing="beta")
df_vol = EmpiricalCopula.to_marginals(emp_cop.random(10000, seed=10), df_emp)

In [ ]:
# def simplified_simulation(train_data_nat_,start_period,end_period): 
#     train_data_nat = train_data_nat_.copy()
#     train_data =train_data_nat.loc[start_period:end_period]
#     model = VAR(train_data)
#     results = model.fit(1)
#     X_test = results.forecast(train_data.values[-1:], steps=1)
#     random_int_vol = random.randint(0, 9999)
#     predicted_l0 = X_test[0][0]
#     onesteppres = []
#     for pc in avg_3_pivot.columns:
#         traningY = df_l.loc[start_period:end_period][pc]
#         trainingdf1 = PC_l0.loc[start_period:end_period].reset_index(drop=True)
#         Y = traningY.values
#         X = trainingdf1.values
#         X_train = sm.add_constant(X)
#         model_full = sm.OLS(Y,X_train)
#         reg_full = model_full.fit()
#         predicted_l2 = reg_full.predict(X_test) 
#         onesteppre =  predicted_l0 + predicted_l2[0] + df_vol.loc[random_int_vol][pc] 
#         onesteppres.append(onesteppre)
#     new_nat_rf = X_test[0]
#     new_nat_rf[0] = np.mean(onesteppres)
#     return new_nat_rf

In [ ]:
# def simplified_simulation_lvl2(train_data_nat):
#     full_data_nat  = train_data_nat.copy() 
#     train_data_nat_ = train_data_nat.copy() 
#     start_period = pd.Period('2007-05', freq='M')
#     end_period = pd.Period('2021-05', freq='M')
#     test_period = pd.Period('2021-06', freq='M')
#     while test_period <  pd.Period('2056-06', freq='M'):
#         new_nat_rf_ = simplified_simulation(train_data_nat_,start_period,end_period) 
#         df_nat_rf = pd.DataFrame([new_nat_rf_], columns =train_data_nat_.columns)
#         full_data_nat = pd.concat([full_data_nat, df_nat_rf],axis = 0)
#         start_period += 1
#         end_period += 1
#         # Update test period by one month
#         test_period += 1
#     return full_data_nat   
        

In [ ]:
# df_onesteperror_nat = pd.DataFrame({},columns = avg_3_pivot.columns)

#     start_period = pd.Period('2005Q1', freq='Q')
#     end_period = pd.Period('2017Q3', freq='Q')
#     test_period = pd.Period('2017Q4', freq='Q')
#     # Define a list to store the test dates
#     errors = [] 
#     # Repeat the simulation for subsequent test dates
#     while test_period <=  pd.Period('2021Q1', freq='Q'):
#         train_data =train_data_nat.loc[start_period:end_period]
#         model = VAR(train_data)
#         results = model.fit(1)
#         X_test = results.forecast(train_data.values[-1:], steps=1)
#         predicted_l0 = X_test[0][0]
#         traningY = df_l.loc[start_period:end_period][pc]
#         trainingdf1 = PC_l0.loc[start_period:end_period].reset_index(drop=True)
#         Y = traningY.values
#         X = trainingdf1.values
#         X_train = sm.add_constant(X)
#         model_full = sm.OLS(Y,X_train)
#         reg_full = model_full.fit()
#         predicted_l2 = reg_full.predict(X_test) 
#         error = df_l.loc[test_period][pc]+train_data_nat.loc[test_period]['l0'] - (predicted_l0 + predicted_l2[0] )
#         errors.append(error)
#         start_period += 1
#         end_period += 1
#         # Update test period by one month
#         test_period += 1
#     df_onesteperror[pc] = errors

In [ ]:
mat_nat = train_data_nat.values
mat_l2 =(df_l.values)[:,1: ]
selected_columns = [0,1,2,3,4,7]
train_data =mat_nat[- 4*16:,selected_columns]
model = VAR(train_data)
VAR_nat = model.fit(1)

In [ ]:
# best_aic = float('inf')
# best_model = None
# for i in range(1, 5):
#     model = VAR(train_data)
#     results = model.fit(i)
#     print( results.aic)
#     if results.aic < best_aic:
#         best_aic = results.aic
#         best_model = results
# print(f'Best model has {best_model.k_ar} lags with AIC of {best_aic}')

# best_bic = float('inf')
# best_model = None
# for i in range(1, 5):
#     model = VAR(train_data)
#     results = model.fit(i)
#     print( results.bic)
#     if results.bic < best_bic:
#         best_bic = results.bic
#         best_model = results
# print(f'Best model has {best_model.k_ar} lags with BIC of {best_bic}')

In [ ]:
jb_test = sm.stats.stattools.jarque_bera(VAR_nat.resid)
print('Jarque-Bera test statistic:', jb_test[0])
print('p-value:', jb_test[1])
# sns.distplot(VAR_nat.resid[:,5])

In [ ]:
# p_threshold = 0.05
# for i in range(len(VAR_nat.params)):
#     for j in range(len(VAR_nat.params[i])):
#         if VAR_nat.pvalues[i][j] > p_threshold:
#             VAR_nat.params[i][j] = 0

In [ ]:
df_emp = pd.DataFrame(VAR_nat.resid)
u = pseudo_obs(df_emp)
emp_cop = EmpiricalCopula(u, smoothing="beta")
df_vol_nat = EmpiricalCopula.to_marginals(emp_cop.random(10000, seed=10), df_emp)

In [ ]:
X = mat_nat[- 4*16:,selected_columns[1:]]
Dict_reg = {}
pc_index = 0 

for pc in avg_3_pivot.columns:  
    Y = mat_l2[- 4*16:,pc_index]
    pc_index = pc_index + 1
    X_train = sm.add_constant(X)
    model_full = sm.OLS(Y,X_train)
    reg_full = model_full.fit()
    Dict_reg[pc] =  reg_full

In [ ]:
# def simplified_simulation_lvl1(mat_nat0,mat_l20): 
#     random_int_vol = random.randint(0, 9999)
#     start_period = - 4*16
#     train_data =mat_nat0[start_period:,:]
#     results = VAR_nat
#     X_test = results.forecast(train_data[-1:], steps=1)
# #     predicted_l0 = X_test[0][0]
    
#     mat_pc_nat = mat_nat0[start_period:,1:]

#     onesteppres = []
#     pc_index = 0 
#     for pc in avg_3_pivot.columns:
# #       arr_l2 = (df_l[pc].values)[start_period:]
# #         Y =mat_l2[start_period:,pc_index]
#         pc_index = pc_index + 1

# #         X_train = sm.add_constant(X)
# #         model_full = sm.OLS(Y,X_train)
# #         reg_full = model_full.fit()
#         reg_full = Dict_reg[pc] 
#         predicted_l2 = reg_full.predict(X_test) 
#         onesteppre =  predicted_l0 + predicted_l2[0] + df_vol.loc[random_int_vol][pc] 
#         onesteppres.append(onesteppre)
   
#     idx = np.abs(df_vol_nat.iloc[:,0] -  df_vol.loc[random_int_vol].mean()).argmin()
# # Extract the whole row
#     closest_row =   df_vol_nat.iloc[idx]
#     new_nat_rf = X_test[0] + closest_row.values
#     return new_nat_rf, onesteppres

In [ ]:
def simplified_simulation_lvl1(mat_nat0,mat_l20): 
    random_int_vol = random.randint(0, 9999)
    start_period = - 4*16
    train_data = mat_nat0[start_period:,:]
##    
#     model = VAR(train_data)
#     results = model.fit(1)
    results = VAR_nat
#     X_test = results.forecast(train_data.values[-1:], steps=1)

    X_test = results.forecast(train_data[-1:], steps=1)
#     predicted_l0 = X_test[0][0]
    
    mat_pc_nat = mat_nat0[start_period:,1:]

    onesteppres = []
    pc_index = 0 
    for pc in avg_3_pivot.columns:      
#       arr_l2 = (df_l[pc].values)[start_period:]
#       
#         X = mat_pc_nat 
#         Y = mat_l20[start_period:,pc_index]    
        pc_index = pc_index + 1       
#         X_train = sm.add_constant(X)
#         model_full = sm.OLS(Y,X_train)
#         reg_full = model_full.fit()    
        reg_full = Dict_reg[pc] 
        predicted_l2 = reg_full.predict(np.insert(X_test[0][1:],0,1)) 
#        onesteppre =  predicted_l0 + predicted_l2[0] + df_vol.loc[random_int_vol][pc] 
        onesteppre =  predicted_l0 + predicted_l2[0] 
        onesteppres.append(onesteppre)
#     idx = np.abs(df_vol_nat.iloc[:,0] -  df_vol.loc[random_int_vol].mean()).argmin()
# Extract the whole row
#     closest_row =   df_vol_nat.iloc[idx]
    new_nat_rf = X_test[0] + df_vol_nat.iloc[random_int_vol]
#     new_nat_rf = X_test[0]
    return new_nat_rf, onesteppres

In [ ]:
def simplified_simulation_lvl2(train_data_nat,df_l):
    mat_nat_ = train_data_nat.values[- 4*16:,:].copy()
    mat_l2_ =(df_l.values)[- 4*16:,1: ].copy()
    sim_step = 0
    while sim_step < 4*35:
        sim_step =  sim_step +1 
        new_nat_rf_, onesteppres_ = simplified_simulation_lvl1(mat_nat_,mat_l2_)
        mat_nat_ = np.vstack([mat_nat_, new_nat_rf_])
        mat_l2_ = np.vstack([mat_l2_,np.array(onesteppres_)])
    return mat_nat_[64:], mat_l2_[64:]

In [ ]:
Dict_sim = {}
for i in range(0,10): 
    mat_nat_, mat_l2_ = simplified_simulation_lvl2(train_data_nat.iloc[:,selected_columns],df_l)
    Dict_sim[i] = mat_nat_
    print(i)
    

In [ ]:
# def rates_ori(arr,ini):
#     arr_ori = []
#     arr_ori.append(ini + arr[0])
#     for i in range(1, np.shape(arr)[0]):
#         arr_ori.append(arr_ori[i-1]+ arr[i])
#     return np.exp(arr_ori)

In [ ]:
# def rates_ori2(arr,a,b):
#     arr_ori = []
#     a0 = arr[0] - a + 2*b
#     arr_ori.append(a0)
#     a1 = arr[1] - b + 2*arr_ori[0]
#     arr_ori.append(a1)
#     for i in range(2, np.shape(arr)[0]):
#         an = arr[i] - arr_ori[i-2] + 2*arr_ori[i-1]
#         arr_ori.append(an)
#     return np.exp(arr_ori)

In [ ]:
narray = Dict_sim[2]
# Extract the first column and name it narr1
narr1 = narray[:, 0]
# Extract the rest of the array and name it narr2

cumulative_growth = np.exp(sum(narr1))-1
cumulative_growth 


In [ ]:
narr1

In [ ]:
narr2 = narray[:, 1:]

# create a numpy array of zeros with the same number of rows as narr2
zeros = np.zeros((narr2.shape[0], 2))

# concatenate zeros to the end of narr2 along axis 1
narr2 = np.concatenate((narr2, zeros), axis=1)


In [ ]:
narr2

In [ ]:

var_nat_processed = pca_nat.inverse_transform(narr2)
var_nat = scaler_pca_nat.inverse_transform(var_nat_processed)
# ###
# var_nat = narr2
ir = var_nat[:,0] 
dvmr = var_nat[:,7] 
ir_ori_ = rates_ori2(ir,macvar_log['ir'].values[-2],macvar_log['ir'].values[-1])
dvmr_ori_ = rates_ori(dvmr,macvar_log['dvmr'].values[-1])

In [ ]:
dvmr_ori_ 

In [ ]:
ir_ori_

In [ ]:
train_data_nat

In [ ]:
 macvar_log['ir'].values[-1]

In [ ]:
array_gr = []
# Dict_exo_rf = {}

# for i in range(0,100):
#     narray = Dict_sim[i]
#     # Extract the first column and name it narr1
#     narr1 = narray[:, 0]
#     cumulative_growth = np.exp(sum(narr1))-1
#     array_gr.append(cumulative_growth)
    
#     # Extract the rest of the array and name it narr2
#     narr2 = narray[:, 1:]
#     var_nat_processed = pca_nat.inverse_transform(narr2)
#     var_nat = scaler_pca_nat.inverse_transform(var_nat_processed)